# Applying YOLOv5s for Traffic Sign Recognition - Train Data Augmenter

This is the code used to augment train data to simulate real-world images

In [1]:
import cv2
import os
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
img_folder = "data-yolo/images/train"
label_folder = "data-yolo/labels/train"

In [3]:
# function to place the sign in a random place on a canvas
def resize_and_place(image, target_size=128):
    # choose a random scale (size of the sign relative to image size)
    h, w = image.shape[:2]
    scale = np.random.uniform(0.05, 0.5)
    new_w = int(target_size * scale)
    new_h = min(target_size - 1, int((new_w / w) * h))
    
    # resize the sign
    resized = cv2.resize(image, (new_w, new_h))

    # place the sign randomly on a blank canvas
    background = np.ones((target_size, target_size, 3), dtype=np.uint8) * 128  
    x_offset = np.random.randint(0, target_size - new_w)
    y_offset = np.random.randint(0, target_size - new_h)
    background[y_offset : y_offset + new_h, x_offset : x_offset + new_w] = resized

    # save normalized coordinates of the center of the image
    x_center = (x_offset + new_w // 2) / target_size
    y_center = (y_offset + new_h // 2) / target_size

    # save normalized width and height
    bbox_width = new_w / target_size
    bbox_height = new_h / target_size
    
    return background, x_center, y_center, bbox_width, bbox_height

In [4]:
train_images = os.listdir(img_folder)
selected_for_augmentation, _ = train_test_split(train_images, test_size=0.5, random_state=42)

In [5]:
for img_name in tqdm(selected_for_augmentation):
    # read image
    img_path = os.path.join(img_folder, img_name)
    img = cv2.imread(img_path)

    # write the augmented image
    augmented_img, x_center, y_center, bbox_width, bbox_height = resize_and_place(img)
    cv2.imwrite(img_path, augmented_img)

    # write the corresponding label
    label_name = img_name.replace('.png', '.txt')
    label_path = os.path.join(label_folder, label_name)
    with open(label_path, "r") as f:
        class_label = f.readline().strip().split()[0]
    
    with open(label_path, "w") as f:
        f.write(f"{class_label} {x_center} {y_center} {bbox_width} {bbox_height}\n")

100%|████████████████████████████████████████████████████████████████████████████| 38700/38700 [07:29<00:00, 86.14it/s]
